In [1]:
import pandas as pd
import numpy as np
from statistics import mode

In [2]:
# Load data 
df = pd.read_csv('data.csv')
df.head()

,3.6216,8.6661,-2.8073,-0.44699,0
0,4.54590,8.1674,-2.4586,-1.46210,0
1,3.86600,-2.6383,1.9242,0.10645,0
2,3.45660,9.5228,-4.0112,-3.59440,0
3,0.32924,-4.4552,4.5718,-0.98880,0
4,4.36840,9.6718,-3.9606,-3.16250,0


In [33]:
def gini_index(groups, classes):
	# count all samples at split point
	n_instances = float(sum([len(group) for group in groups]))
	# sum weighted Gini index for each group
	gini = 0.0
	for group in groups:
		size = float(len(group))
		# avoid divide by zero
		if size == 0:
			continue
		score = 0.0
		# score the group based on the score for each class
		for class_val in classes:
			p = [row[-1] for row in group].count(class_val) / size
			score += p * p
		# weight the group score by its relative size
		gini += (1.0 - score) * (size / n_instances)
	return gini

In [4]:

# Split a dataset based on an attribute and an attribute value
def test_split(index, value, dataset):
	left, right = list(), list()
	for row in dataset:
		if row[index] < value:
			left.append(row)
		else:
			right.append(row)
	return left, right

In [54]:
def get_split(dataset):
    classes = list(set(row[-1] for row in dataset))
    attributes = len(dataset[0]) - 1
    retAttribute,retValue, retGroups, min_gini = None, None, None, 999
    for row in dataset:
        for attribute in range(attributes):
            groups = test_split(attribute, row[attribute], dataset)
            gini = gini_index(groups, classes)
            if gini < min_gini:
                min_gini = gini
                retAttribute, retValue, retGroups = attribute, row[attribute], groups
    return {'Attribute':retAttribute, 'Value': retValue, 'Groups': retGroups}
get_split(dataset)

{'Attribute': 0,
 'Value': 6.642287351,
 'Groups': ([[2.771244718, 1.784783929, 0],
   [1.728571309, 1.169761413, 0],
   [3.678319846, 2.81281357, 0],
   [3.961043357, 2.61995032, 0],
   [2.999208922, 2.209014212, 0]],
  [[7.497545867, 3.162953546, 1],
   [9.00220326, 3.339047188, 1],
   [7.444542326, 0.476683375, 1],
   [10.12493903, 3.234550982, 1],
   [6.642287351, 3.319983761, 1]])}

In [55]:

def to_terminal(group):
	outcomes = [row[-1] for row in group]
	return mode(outcomes)  #most frequency
group = [[1,2,0],[1,2,1],[1,2,1]]
to_terminal(group)

1

In [ ]:
len(dataset[0])

3

In [17]:
dataset = [[2.771244718,1.784783929,0],
	[1.728571309,1.169761413,0],
	[3.678319846,2.81281357,0],
	[3.961043357,2.61995032,0],
	[2.999208922,2.209014212,0],
	[7.497545867,3.162953546,1],
	[9.00220326,3.339047188,1],
	[7.444542326,0.476683375,1],
	[10.12493903,3.234550982,1],
	[6.642287351,3.319983761,1]]